# Calculador de Sazonalidade Mensal da Receita Prevista

Este Jupyter Notebook calcula a sazonalidade mensal de determinada receita de acordo com as arrecadações mensais em anos anteriores.

## Procedimentos de cálculo

Data uma determinada receita, determinado valor anual previsto e determinado intervalo de anos anteriores:

1. Para cada ano anterior, identifica o valor mensal arrecadado e o total arrecadado no ano;
2. Para cada ano anterior, calcula o peso percentual da arrecadação mensal em relação ao total arrecadado;
3. Para cada mês, apura a média do peso percentual arrecadado;
4. Tendo por base o valor anual previsto, para cada mês aplica o percentual médio ponderado a fim de obter a previsão mensal da receita.

## Especificidades do cálculo

### Arredondamento dos valores mensais previstos

Foi adotado um coeficiente de arredondamento para ser usado nos valores previstos para janeiro a novembro, sendo a diferença acumulada para o mês de dezembro.

## Origem dos dados

Os dados utilizados são os do arquivo RECEITA.TXT gerado para o SIAPC/PAD do TCE/RS convertidos pelo [pad_converter](https://github.com/iddrs/pad-converterhttps://github.com/iddrs/pad-converter).

In [ ]:
# Configurando o ambiente

import pandas as pd
from os import path

'''
Diretório base para os dados.

Dentro desse diretório deve haver um subdiretório para cada ano especificado em ```anos_anteriores``` (definida no bloco seguinte)

Dentro de cada subdiretório é necessário constar um arquivo *RECEITA.CSV*.
'''
dir_base_dados = r'./dados'


## Configuração geral do cálculo

In [ ]:
# Configurando as bases do cálculo

'''
Lista dos anos que serão considerados para o cálculo dos percentuais.

Obviamente que presia ser um intervalo constante de anos. Então, nada de ficar "pulando" anos por aí.

Além disso, para cada ano é necessário existir um diretório dentro de *dados* e dentro deles, um arquivo *RECEITA.CSV*
'''
anos_anteriores = (2019, 2020, 2021)

'''
O valor de ```ndigits``` da [função built-in](https://docs.python.org/3/library/functions.html#round) do Python ```round()```.
'''
coeficiente_arredondamento = -3

## Carga dos dados

In [ ]:
# Executando o cálculo
# Não mude nada a partir daqui, exceto se souber o que estás fazendo

# Carrega os dados de receita

# Lista com as colunas a retornar no dataframe
colunas_nomes = [
    'codigo_receita',
    'receita_realizada_jan',
    'receita_realizada_fev',
    'receita_realizada_mar',
    'receita_realizada_abr',
    'receita_realizada_mai',
    'receita_realizada_jun',
    'receita_realizada_jul',
    'receita_realizada_ago',
    'receita_realizada_set',
    'receita_realizada_out',
    'receita_realizada_nov',
    'receita_realizada_dez'
]

# Armazena os dataframes com os dados das receitas
dados_receita = {}
for ano in anos_anteriores:
    f = path.join(dir_base_dados, str(ano), 'RECEITA.csv')
    print(f'Carregando dados de {ano} a partir de {f}...')
    df = pd.read_csv(f, sep=';', usecols=colunas_nomes, thousands='.', decimal=',')
    dados_receita[ano] = df

## Configuração das receitas

In [ ]:
# Configuração de/para dos códigos da receita

'''
Graças às mudanças mais frequentes do que gostaríamos na codificação da receita (obrigado STN, obrigado TCE/RS), é necessário fazer um de/para nos códigos das receitas.

A estrutura do de/para é a seguinte:

- Dicionário onde chave = identificador da receita e valor = lista
    - Lista
        - String com um identificador (nome) da receita
        - Dicionário onde chave = ano e valor = código da receita naquele ano
'''

de_para = {
    'iptu_principal': ['IMPOSTO SOBRE A PROPRIEDADE PREDIAL E TERRITORIAL URBANA - PRINCIPAL',
    {
        2019: '4.1.1.1.8.01.1.1.00.00.00.00.000',
        2020: '4.1.1.1.8.01.1.1.00.00.00.00.000',
        2021: '4.1.1.1.8.01.1.1.00.00.00.00.000'
    }],
    'iptu_mjm': ['IMPOSTO SOBRE A PROPRIEDADE PREDIAL E TERRITORIAL URBANA - MULTA E JUROS DE MORA',
    {
        2019: '4.1.1.1.8.01.1.2.00.00.00.00.000',
        2020: '4.1.1.1.8.01.1.2.00.00.00.00.000',
        2021: '4.1.1.1.8.01.1.2.00.00.00.00.000'
    }],
    'irrf_trabalho_exec': ['IRRF SOBRE RENDIMENTOS DO  TRABALHO - PRINCIPAL  - ATIVOS/INATIVOS DO PODER EXECUTIVO/INDIRETAS',
    {
        2019: '4.1.1.1.3.03.1.1.01.00.00.00.000',
        2020: '4.1.1.1.3.03.1.1.01.00.00.00.000',
        2021: '4.1.1.1.3.03.1.1.01.00.00.00.000'
    }],
    'irrf_trabalho_leg': ['IRRF SOBRE RENDIMENTOS DO  TRABALHO - PRINCIPAL  - ATIVOS/INATIVOS DO PODER LEGISLATIVO',
    {
        2019: '4.1.1.1.3.03.1.1.02.00.00.00.000',
        2020: '4.1.1.1.3.03.1.1.02.00.00.00.000',
        2021: '4.1.1.1.3.03.1.1.02.00.00.00.000'
    }],
    'irrf_trabalho_rpps': ['IRRF SOBRE RENDIMENTOS DO  TRABALHO - PRINCIPAL  - INATIVOS PAGOS PELO RPPS',
    {
        2019: '4.1.1.1.3.03.1.1.03.00.00.00.000',
        2020: '4.1.1.1.3.03.1.1.03.00.00.00.000',
        2021: '4.1.1.1.3.03.1.1.03.00.00.00.000'
    }],
    'issqn': ['IMPOSTO SOBRE SERVICOS DE QUALQUER NATUREZA - PRINCIPAL',
    {
        2019: '4.1.1.1.8.02.3.1.00.00.00.00.000',
        2020: '4.1.1.1.8.02.3.1.00.00.00.00.000',
        2021: '4.1.1.1.8.02.3.1.00.00.00.00.000'
    }],
    'issqn_mjm': ['IMPOSTO SOBRE SERVICOS DE QUALQUER NATUREZA - MULTA E JUROS DE MORA',
    {
        2019: '4.1.1.1.8.02.3.2.00.00.00.00.000',
        2020: '4.1.1.1.8.02.3.2.00.00.00.00.000',
        2021: '4.1.1.1.8.02.3.2.00.00.00.00.000'
    }],
    'contrib_rpps_ativo': ['CPSSS DO SERVIDOR CIVIL ATIVO - PRINCIPAL',
    {
        2019: '4.1.2.1.8.01.1.1.00.00.00.00.000',
        2020: '4.1.2.1.8.01.1.1.00.00.00.00.000',
        2021: '4.1.2.1.8.01.1.1.00.00.00.00.000'
    }],
    'contrib_rpps_inativo': ['CPSSS DO SERVIDOR CIVIL INATIVO - PRINCIPAL',
    {
        2019: '4.1.2.1.8.01.2.1.00.00.00.00.000',
        2020: '4.1.2.1.8.01.2.1.00.00.00.00.000',
        2021: '4.1.2.1.8.01.2.1.00.00.00.00.000'
    }],
    'contrib_rpps_pensionista': ['CPSSS DO SERVIDOR CIVIL - PENSIONISTAS - PRINCIPAL',
    {
        2019: '4.1.2.1.8.01.3.1.00.00.00.00.000',
        2020: '4.1.2.1.8.01.3.1.00.00.00.00.000',
        2021: '4.1.2.1.8.01.3.1.00.00.00.00.000'
    }],
    'cip': ['CONTRIBUICAO PARA O CUSTEIO DO SERVICO DE ILUMINACAO PUBLICA - PRINCIPAL',
    {
        2019: '4.1.2.4.0.00.1.1.00.00.00.00.000',
        2020: '4.1.2.4.0.00.1.1.00.00.00.00.000',
        2021: '4.1.2.4.0.00.1.1.00.00.00.00.000'
    }],
    'fpm': ['COTA-PARTE DO FUNDO DE PARTICIPACAO DOS MUNICIPIOS - COTA MENSAL - PRINCIPAL',
    {
        2019: '4.1.7.1.8.01.2.1.00.00.00.00.000',
        2020: '4.1.7.1.8.01.2.1.00.00.00.00.000',
        2021: '4.1.7.1.8.01.2.1.00.00.00.00.000'
    }],
    'itr': ['COTA-PARTE DO IMPOSTO SOBRE A PROPRIEDADE TERRITORIAL RURAL - PRINCIPAL',
    {
        2019: '4.1.7.1.8.01.5.1.00.00.00.00.000',
        2020: '4.1.7.1.8.01.5.1.00.00.00.00.000',
        2021: '4.1.7.1.8.01.5.1.00.00.00.00.000'
    }],
    'icms': ['COTA-PARTE DO ICMS - PRINCIPAL',
    {
        2019: '4.1.7.2.8.01.1.1.00.00.00.00.000',
        2020: '4.1.7.2.8.01.1.1.00.00.00.00.000',
        2021: '4.1.7.2.8.01.1.1.00.00.00.00.000'
    }],
    'ipva': ['COTA-PARTE DO IPVA - PRINCIPAL',
    {
        2019: '4.1.7.2.8.01.2.1.00.00.00.00.000',
        2020: '4.1.7.2.8.01.2.1.00.00.00.00.000',
        2021: '4.1.7.2.8.01.2.1.00.00.00.00.000'
    }],
    'ipi': ['COTA-PARTE DO IPI - MUNICIPIOS - PRINCIPAL',
    {
        2019: '4.1.7.2.8.01.3.1.00.00.00.00.000',
        2020: '4.1.7.2.8.01.3.1.00.00.00.00.000',
        2021: '4.1.7.2.8.01.3.1.00.00.00.00.000'
    }],
    'fundeb': ['TRANSFERENCIAS DE RECURSOS DO FUNDO DE MANUTENCAO E DESENVOLVIMENTO DA EDUCACAO BASICA E DE VALORIZACAO DOS PROFISSIONAIS DA EDUCACAO – FUND',
    {
        2019: '4.1.7.5.8.01.1.1.00.00.00.00.000',
        2020: '4.1.7.5.8.01.1.1.00.00.00.00.000',
        2021: '4.1.7.5.8.01.1.1.00.00.00.00.000'
    }],
    'stt': ['PROGRAMA TROCA-TROCA - PRINCIPAL',
    {
        2019: '4.1.9.2.8.02.9.1.02.00.00.00.000',
        2020: '4.1.9.2.8.02.9.1.02.00.00.00.000',
        2021: '4.1.9.2.8.02.9.1.02.00.00.00.000'
    }],
    'patronal_normal_ativos': ['CPSSS PATRONAL - SERVIDOR CIVIL ATIVO - PRINCIPAL',
    {
        2019: '4.7.2.1.8.03.1.1.00.00.00.00.000',
        2020: '4.7.2.1.8.03.1.1.00.00.00.00.000',
        2021: '4.7.2.1.8.03.1.1.00.00.00.00.000'
    }],
    'patronal_suplementar_ativos': ['CPSSS PATRONAL - AMORTIZACAO DO PASSIVO ATUARIAL - PRINCIPAL',
    {
        2019: '4.7.2.1.8.03.1.1.02.00.00.00.000',
        2020: '4.7.2.1.8.03.1.1.02.00.00.00.000',
        2021: '4.7.2.1.8.03.1.1.02.00.00.00.000'
    }],
    'patronal_normal_inativos': ['CPSS PATRONAL - SERVIDOR CIVIL INATIVO - PRINCIPAL INTRA-ORCAMENTARIA',
    {
        2019: '4.7.2.1.8.03.2.1.00.00.00.00.000',
        2020: '4.7.2.1.8.03.2.1.00.00.00.00.000',
        2021: '4.7.2.1.8.03.2.1.00.00.00.00.000'
    }],
    'patronal_normal_pensionista': ['CPSSS PATRONAL - SERVIDOR CIVIL - PENSIONISTAS - PRINCIPAL',
    {
        2019: '4.7.2.1.8.03.3.1.00.00.00.00.000',
        2020: '4.7.2.1.8.03.3.1.00.00.00.00.000',
        2021: '4.7.2.1.8.03.3.1.00.00.00.00.000'
    }],
    'parcelamento_divida_rpps': ['CPSSS PATRONAL - PARCELAMENTOS - SERVIDOR CIVIL ATIVO - PRINCIPAL',
    {
        2019: '4.7.2.1.8.04.1.1.00.00.00.00.000',
        2020: '4.7.2.1.8.04.1.1.00.00.00.00.000',
        2021: '4.7.2.1.8.04.1.1.00.00.00.00.000'
    }],
}

## Configuração do cálculo

In [ ]:
# Configurando o cálculo

# A chave da receita alvo usado em ```de_para```
receita_alvo = 'patronal_normal_ativos'
# Valor previsto para a receita alvo. É o valor que queremos dividir de acordo com a sazonalidade mensal
valor_receita_alvo = 11000


## Cálculo

In [ ]:
# Pega a especificação da receita alvo
especificacao_receita_alvo = de_para[receita_alvo]

### Busca os valores arrecadados em cada ano

In [ ]:
# Busca os valores arrecadados para cada ano

# Armazena a arrecadação mensal e total da receita alvo
arrecadacao = {}
for ano in anos_anteriores:
    codigo_receita = especificacao_receita_alvo[1][ano]
    df = dados_receita[ano]
    filtro = df[df['codigo_receita'] == codigo_receita].copy()
    filtro['total'] = filtro['receita_realizada_jan'] + filtro['receita_realizada_fev'] + filtro['receita_realizada_mar'] + filtro['receita_realizada_abr'] + filtro['receita_realizada_mai'] + filtro['receita_realizada_jun'] + filtro['receita_realizada_jul'] + filtro['receita_realizada_ago'] + filtro['receita_realizada_set'] + filtro['receita_realizada_out'] + filtro['receita_realizada_nov'] + filtro['receita_realizada_dez']
    arrecadacao[ano] = filtro

### Calcula o percentual mensal arrecadado

In [ ]:
# Calcula o percentual mensal de cada ano

# Nomes das colunas mensais
meses = colunas_nomes[1:].copy()

# Armazena os percentuais mensais de cada ano
percentuais = {}
for ano in anos_anteriores:
    percentuais[ano] = {}
    arrecadado = arrecadacao[ano]
    total = float(arrecadado['total'])
    acumulado = 0.0
    for mes in meses:
        percentuais[ano][mes] = {}
        if mes == 'receita_realizada_dez':
            percentuais[ano][mes] = total - acumulado
        else:
            percentuais[ano][mes] = float(arrecadado[mes]) / total
            acumulado += arrecadado[mes]
            
# Calcula a média mensal
percentuais_medios = {}
perc_total = 0.0
for mes in meses:
    perc_acum = 0.0
    for ano in anos_anteriores:
        perc_acum += float(percentuais[ano][mes])
    percentuais_medios[mes] = perc_acum / len(anos_anteriores)
    perc_total += perc_acum

### Calcula os valroes previstos sazonalizados

In [ ]:
# Calcula os valores mensais da receita alvo

receita_acum = 0.0
receita_alvo_sazonalizada = {}
for mes in meses:
    perc = percentuais_medios[mes]
    if mes == 'receita_realizada_dez':
        receita_alvo_sazonalizada[mes] = valor_receita_alvo - receita_acum
        receita_acum += receita_alvo_sazonalizada[mes]
    else:
        receita_alvo_sazonalizada[mes] = round(valor_receita_alvo * perc, coeficiente_arredondamento)
        receita_acum += receita_alvo_sazonalizada[mes]

## Resultado

In [ ]:
# Mostra/salva o resultado
pd.options.display.float_format = '{:,.2f}'.format
df = pd.DataFrame(list(receita_alvo_sazonalizada.items()), columns=['Mês', 'Valor'])
df['20% Fundeb'] = round(df.Valor * 0.2, 2)
print(df)
'''
Eu incluí uma coluna com 20% do valor mensal para facilitar no caso de receitas que tem dedução para o Fundeb.
'''